In [49]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split


import warnings;
warnings.filterwarnings('ignore');

### Dataset
<hr>

In [35]:
import pandas as pd
import pandas as pd

df = pd.read_csv("result/01_date_preprocessing_result.csv",sep=",")
df.head()

,Complaint_No,narrative,narrative_length,credit_card,credit_reporting,debt_collection,mortgages_and_loans,retail_banking,Complaint_tokenized_text
0,0,purchase order day shipping amount receive pro...,1705,True,False,False,False,False,"[['purchase', 'order', 'day', 'shipping', 'amo..."
1,1,forwarded message date tue subject please inve...,904,True,False,False,False,False,"[['forwarded', 'message', 'date', 'tue', 'subj..."
2,2,forwarded message cc sent friday pdt subject f...,1230,False,False,False,False,True,"[['forwarded', 'message', 'cc', 'sent', 'frida..."
3,3,payment history missing credit report speciali...,903,False,True,False,False,False,"[['payment', 'history', 'missing', 'credit', '..."
4,4,payment history missing credit report made mis...,851,False,True,False,False,False,"[['payment', 'history', 'missing', 'credit', '..."


### Splitting Data

In [36]:
X = df['narrative'].values

In [37]:
y = np.argmax(df[['credit_card', 'credit_reporting', 'debt_collection', 'mortgages_and_loans', 'retail_banking']].values, axis=1)

In [38]:
y.shape

(162411,)

In [39]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2 , random_state=22)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [40]:
X_train.shape

(103942,)

In [41]:
y_train.shape

(103942,)

In [42]:
X_test.shape

(32483,)

In [43]:
y_test.shape

(32483,)

### Modeling

In [56]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN,  Embedding, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [57]:
# Defining training parameters
max_sequence_length = 170   
max_words = 2500   
embedding_dim = 32  
# Function to build the CNN model with one hidden layer
def build_model():
    model = Sequential()
    
    model.add(Embedding(max_words, embedding_dim, input_length=max_sequence_length))
    model.add(SimpleRNN(units=50, return_sequences = True))
    model.add(Dropout(0.2))

    model.add(SimpleRNN(units=50, return_sequences = True))
    model.add(Dropout(0.2))

    model.add(SimpleRNN(units=50, return_sequences = True))
    model.add(Dropout(0.2))

    model.add(SimpleRNN(units=50))
    model.add(Dropout(0.2))
   
    return model

In [58]:
# Build the CNN model
model = build_model()
# Menampilkan ringkasan model
model.summary()

ValueError: Unrecognized keyword arguments passed to Embedding: {'input_length': 170}

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=30, batch_size=100, validation_data=(X_train, y_train))

In [ ]:
# Ambil loss dari history
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(train_loss) + 1)

In [ ]:
# Plot kurva loss
plt.plot(epochs, train_loss, 'g', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Mengevaluasi kinerja model menggunakan data validasi
val_loss, val_acc = model.evaluate(X_val, y_val)
print("Model Validation Accuracy:", val_acc)

### Evaluating the Model

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test,y_test)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')